In [14]:
!pip install --upgrade transformers
!pip install transformers[torch]
!pip install 'accelerate>=0.26.0'


zsh:1: no matches found: transformers[torch]


In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import transformers
print(transformers.__version__)


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.51.3


In [3]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Cargar datos limpios
df = pd.read_csv("../../../../data/processed/train_preprocess_v1.csv")
df = df[['statement', 'label']]  # Asegurarse de usar solo lo necesario

# División
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(example):
    return tokenizer(example['statement'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# Eliminar columnas no necesarias
train_dataset = train_dataset.remove_columns(["statement", "__index_level_0__"])
val_dataset = val_dataset.remove_columns(["statement", "__index_level_0__"])


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\afgdl.ASUSS15ANGEL\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 1790/1790 [00:01<00:00, 1100.41 examples/s]


In [6]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from sklearn.metrics import f1_score

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=1).numpy()
    return {"f1": f1_score(labels, preds)}

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-test",
    num_train_epochs=1,                     
    per_device_train_batch_size=16,         
    per_device_eval_batch_size=32,          
    learning_rate=5e-5,                      
    weight_decay=0.01,
    eval_strategy="steps",            
    eval_steps=100,                   
    save_strategy="no",               
    logging_steps=50,                 
    disable_tqdm=False,               
    report_to="none"                        
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\afgdl.ASUSS15ANGEL\AppData\Local\Temp\ipykernel_27228\717415926.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()
trainer.evaluate()

c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,F1
100,0.647300,0.648690,0.778978
200,0.653900,0.627033,0.754702
300,0.622200,0.621433,0.742097
400,0.638100,0.617151,0.752679


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memo

{'eval_loss': 0.6142523288726807,
 'eval_f1': 0.7656187823318743,
 'eval_runtime': 99.9895,
 'eval_samples_per_second': 17.902,
 'eval_steps_per_second': 0.56,
 'epoch': 1.0}

In [9]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Evaluar y obtener predicciones sobre val_dataset
pred_out = trainer.predict(val_dataset)

# Sacar predicciones
val_preds = np.argmax(pred_out.predictions, axis=1)

# Etiquetas reales
y_val = val_df['label'].values

# Matriz de confusión
cm = confusion_matrix(y_val, val_preds)
print("Matriz de confusión:\n", cm)

# Reporte completo
print("Reporte de clasificación:\n", classification_report(y_val, val_preds))

# Macro F1
from sklearn.metrics import f1_score
print("Macro F1:", f1_score(y_val, val_preds, average='macro'))


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Matriz de confusión:
 [[239 392]
 [197 962]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.55      0.38      0.45       631
           1       0.71      0.83      0.77      1159

    accuracy                           0.67      1790
   macro avg       0.63      0.60      0.61      1790
weighted avg       0.65      0.67      0.65      1790

Macro F1: 0.606801893508955


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset

# 1. Cargar CSV de test (sin etiquetas)
test_df = pd.read_csv("../../../../data/processed/test_preprocess_v1.csv")  # Cambia la ruta si hace falta

# 2. Crear Dataset HuggingFace
test_dataset = Dataset.from_pandas(test_df)

# 3. Tokenizar el test_dataset igual que el train/val
def tokenize(example):
    return tokenizer(example['statement'], truncation=True, padding='max_length', max_length=128)

test_dataset = test_dataset.map(tokenize, batched=True)

# 4. Eliminar columnas no necesarias para el modelo, sólo si existen
cols_to_remove = ["statement", "__index_level_0__"]
cols_present = [col for col in cols_to_remove if col in test_dataset.column_names]
if cols_present:
    test_dataset = test_dataset.remove_columns(cols_present)

# 5. Predecir
pred_out = trainer.predict(test_dataset)

# 6. Obtener predicciones (clase con mayor probabilidad)
test_preds = np.argmax(pred_out.predictions, axis=1)

# 7. Preparar DataFrame para Kaggle:
if 'id' in test_df.columns:
    submission_ids = test_df['id']
else:
    submission_ids = test_df.index

submission_df = pd.DataFrame({
    "id": submission_ids,
    "label": test_preds
})

# 8. Guardar CSV listo para subir a Kaggle
submission_df.to_csv("submission.csv", index=False)

print("Archivo submission.csv creado con éxito.")


Map: 100%|██████████| 3836/3836 [00:01<00:00, 2244.18 examples/s]
c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Archivo submission.csv creado con éxito.
